<a href="https://www.kaggle.com/code/shravankumar147/hugging-face-models-with-langchain-for-resume-data?scriptVersionId=214450951" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Integrating Hugging Face Models with LangChain for Resume Data Extraction

In [1]:
!pip install transformers langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 9.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00


In [2]:
from transformers import pipeline
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

# Load a Hugging Face model using the pipeline
model_name = "tiiuae/falcon-7b-instruct"  # Replace with your desired Hugging Face model
llm = pipeline("text2text-generation", model=model_name)

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
The model 'FalconForCausalLM' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForCondit

In [3]:
# System Message
system_message_prompt = SystemMessagePromptTemplate.from_template(
    """You are a helpful AI assistant who answers user's questions based on the provided context"""
)

# Prompt Template
prompt = """
            **Task:** Extract key information from the following resume text.

            **Resume Text:**
            {context}

            **Instructions:**
            Please extract the following information and format it in a clear structure:

            1. **Contact Information:**
            - Name:
            - Email:
            - Phone Number:
            - Website/Portfolio/LinkedIn:
            - Github Profile:

            2. **Education:**
            - Institution Name:
            - Degree:
            - Field of Study:
            - Graduation Date:

            3. **Experience:**
            - Job Title:
            - Company Name:
            - Location:
            - Dates of Employment:
            - Responsibilities/Projects:

            4. **Projects:**
            - Project Title:
            - Description/Technologies Used:
            - Outcomes/Results:

            5. **Skills:**
            - Programming Languages:
            - Technologies/Tools/frameworks:

            6. **Additional Information:** (if applicable)
            - Certifications:
            - Awards or Honors:
            - Professional Affiliations:
            - Languages:

            **Question:**
            {question}

            **Extracted Information:**
        """

human_message_prompt = HumanMessagePromptTemplate.from_template(prompt)

# Define a function to call the model
def LLM_chain(context, question):
    template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
    complete_prompt = template.format_messages({"context": context, "question": question})[1].content
    response = llm(complete_prompt, max_length=500)[0]['generated_text']
    return response

# JSON Validation Chain
def LLM_chain_json(context):
    json_prompt_validate = """
            **Task:** Validate and correct the following JSON data:

            **JSON Data:**
            {context}

            **Instructions:**
            Provide JSON-only output with no explanation or preamble.

            **Corrected JSON:**"""
    human_message = HumanMessagePromptTemplate.from_template(json_prompt_validate)
    template = ChatPromptTemplate.from_messages([system_message_prompt, human_message])
    complete_prompt = template.format_messages({"context": context})[1].content
    response = llm(complete_prompt, max_length=500)[0]['generated_text']
    return response

In [ ]:
# !pip install transformers langchain-core

from transformers import pipeline
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

# Load a Hugging Face model
model_name = "tiiuae/falcon-7b-instruct"  # Choose an appropriate model
llm = pipeline("text2text-generation", model=model_name)

In [6]:
# Prompt Templates
system_message_prompt = SystemMessagePromptTemplate.from_template(
    """You are a helpful AI assistant who answers user's questions based on the provided context"""
)
prompt = """
**Task:** Extract key information from the following resume text.

**Resume Text:**
{context}

**Instructions:**
Please extract the following information and format it in a clear structure:

1. **Contact Information:**
- Name:
- Email:
- Phone Number:
- Website/Portfolio/LinkedIn:
- Github Profile:

2. **Education:**
- Institution Name:
- Degree:
- Field of Study:
- Graduation Date:

3. **Experience:**
- Job Title:
- Company Name:
- Location:
- Dates of Employment:
- Responsibilities/Projects:

4. **Projects:**
- Project Title:
- Description/Technologies Used:
- Outcomes/Results:

5. **Skills:**
- Programming Languages:
- Technologies/Tools/frameworks:

**Question:**
{question}

**Extracted Information:**
"""
human_message_prompt = HumanMessagePromptTemplate.from_template(prompt)

def LLM_chain(context, question):
    # Format messages using keyword arguments
    template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
    complete_prompt = template.format_messages(context=context, question=question)[1].content
    response = llm(complete_prompt, max_length=500)[0]['generated_text']
    return response


# Test Data
resume_text = """
John Doe
johndoe@example.com
+1 123-456-7890
linkedin.com/in/johndoe
github.com/johndoe

Education:
- XYZ University, B.Sc. Computer Science, 2015-2019

Experience:
- Software Engineer at ABC Corp, Jan 2020 - Present
  Responsibilities: Developed web applications using Python and React.

Skills:
- Python, JavaScript, React, SQL
"""
question = "Extract key information from this resume."

# Run Extraction
response = LLM_chain(resume_text, question)
print("Extracted Information:\n", response)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Extracted Information:
 
**Task:** Extract key information from the following resume text.

**Resume Text:**

John Doe
johndoe@example.com
+1 123-456-7890
linkedin.com/in/johndoe
github.com/johndoe

Education:
- XYZ University, B.Sc. Computer Science, 2015-2019

Experience:
- Software Engineer at ABC Corp, Jan 2020 - Present
  Responsibilities: Developed web applications using Python and React.

Skills:
- Python, JavaScript, React, SQL


**Instructions:**
Please extract the following information and format it in a clear structure:

1. **Contact Information:**
- Name:
- Email:
- Phone Number:
- Website/Portfolio/LinkedIn:
- Github Profile:

2. **Education:**
- Institution Name:
- Degree:
- Field of Study:
- Graduation Date:

3. **Experience:**
- Job Title:
- Company Name:
- Location:
- Dates of Employment:
- Responsibilities/Projects:

4. **Projects:**
- Project Title:
- Description/Technologies Used:
- Outcomes/Results:

5. **Skills:**
- Programming Languages:
- Technologies/Tools/fram